This file was run on kaggle.
So if it is to be run elsewhere, changes should be made accordingly.

In [ ]:
!pip install gdown

## Downloading required python files
In our case, we have done it by downloading those files from google drive.

In [ ]:
#buildDataset.py
!gdown 1Si--hPPSHC1TtQfC_EM6zDYI4GYLeCdC

#loss_function.py
!gdown 10isYKF9TwYm0wQkHGY3Ngpp76tx8gO-w

#metrics.py
!gdown 1ujvPjTNogAq9ws4CnU_h-Er-o11iJvet

#dataAugmentation.py
!gdown 1NB8caC04dgCAnmaCQI11xgnGv19oMMUb

## Getting Dataset
We have done it by downloading it from google drive.
The script was written for training a model only for a single fold.
Changes should be made accordingly for training it for all the folds at once.

In [ ]:
%cd /kaggle/working
#Downloading training data and validation data respectively

#Fold4: trainging and validation respectively
!gdown 1fTaFl-MPLvjFf3GuDju6kseMzSY-CqOl
!gdown 1K_nDTXQoDK3DNSWJY3oXfdzJY5wzmmeY

In [ ]:
!unzip -q Training.zip -d /kaggle/working
!unzip -q Validation.zip -d /kaggle/working

## Importing all the required python libraries 

In [ ]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import cv2
import math

import tensorflow as tf
import tensorflow_hub as hub

import loss_function
import metrics
import buildDataset

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## Creating the model architecture
The variable do_fine_tuning should be set to True when training for Phase-II

In [ ]:
model_names = ["mobilenet_v3_large_100_224","inception_v3","efficientnetv2-s-21k-ft1k"]

model_handles = ["https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
                 "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5",
                 "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2"
                ]

In [ ]:
#currently using EfficientNetV2S
model_name = model_names[2]
model_handle = model_handles[2]

do_fine_tuning = False
IMAGE_SIZE = (384,384)
totClasses=8
CLASSES=['Curved_Mayo_Scissor','Scalpel','Straight_Dissection_Clamp','Babcock_Tissue_Forceps','Mayo_Needle_Holder','Deaver_Retractor','Metzenbaum_Scissor','Microvascular_Needle_Holder']

In [ ]:
#Model Architecture
inputLayer=tf.keras.Input(shape=IMAGE_SIZE + (3,))
x=hub.KerasLayer(model_handle,trainable=do_fine_tuning)(inputLayer)


b1=tf.keras.layers.Dense(256)(x)
b1Activation=tf.keras.layers.LeakyReLU(alpha=0.1)(b1)
b1D=tf.keras.layers.Dropout(rate=0.2)(b1Activation)
ClassificationLayer=tf.keras.layers.Dense(totClasses,activation='softmax',name='ClassificationLayer')(b1D)


b2=tf.keras.layers.Dense(256)(x)
b2Activation=tf.keras.layers.LeakyReLU(alpha=0.1)(b2)
b2D=tf.keras.layers.Dropout(rate=0.2)(b2Activation)
BBCoordinateLayer=tf.keras.layers.Dense(4,activation='sigmoid',name='BBCoordinateLayer')(b2D)


b3=tf.keras.layers.Dense(256)(x)
b3Activation=tf.keras.layers.LeakyReLU(alpha=0.1)(b3)
b3D=tf.keras.layers.Dropout(rate=0.2)(b3Activation)
AngleLayer=tf.keras.layers.Dense(1,activation='tanh',name='AngleLayer')(b3D)


FinalOutputLayer=tf.keras.layers.Concatenate(name='FinalOutputLayer')([ClassificationLayer,BBCoordinateLayer,AngleLayer])

model=tf.keras.Model(inputs=inputLayer,outputs=[FinalOutputLayer])

In [ ]:
#model.build((None,)+IMAGE_SIZE+(3,))
model.summary()
tf.keras.utils.plot_model(model,show_shapes=True,dpi=96)

## Initializing with best weights from Phase 1
If we are training for Phase 2, then we must initialize the model with appropriate weights. We are downloading the appropriate weights from google drive.

In [ ]:
#weights file
!gdown 1vdBXH4_prcdlciZgL_WFJHF0UCO4SgtL   #Fold4


#Index File
!gdown 1z5Zav-c18s3dQYF5hp1_QCwIxiSY9seR   #Fold4

In [ ]:
model.load_weights("/kaggle/working/best_model.18-0.27")  #Fold4

## Training

In [ ]:
#Configuring the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss=loss_function.MSEAndSCCLoss,
              metrics=[metrics.Caccuracy,metrics.Laccuracy,metrics.Daccuracy],run_eagerly=True
             )

In [ ]:
#building dataset
#converting images and labels to tensors
trainFilesPath='/kaggle/working/Training'
validFilesPath='/kaggle/working/Validation'
trainX,trainY=buildDataset.covert_dataset_to_tensors(trainFilesPath, augmentation = True)
validX,validY=buildDataset.covert_dataset_to_tensors(validFilesPath)

In [ ]:
batch_size=8
epochs=50

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.{epoch:02d}-{val_Daccuracy:.2f}", monitor='val_Daccuracy', verbose=1,
    save_best_only=True, mode='max', period=1,save_weights_only=True)

In [ ]:
history=model.fit(trainX,trainY,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(validX,validY),
          callbacks=[checkpoint]
         )

## Saving the model

In [ ]:
#Saving the model history to a csv file
historyDF=pd.DataFrame(history.history)

historyCSVFile='Model_History.CSV'
with open(historyCSVFile,'w') as file:
    historyDF.to_csv(file)

!mkdir /kaggle/working/SAVED_MODEL_AND_WEIGHTS

In [ ]:
#Saving the complete model including weights, architecture and everything
model.save('/kaggle/working/SAVED_MODEL_AND_WEIGHTS/GSTENV2-S')

#Saving only weights
model.save_weights('/kaggle/working/SAVED_MODEL_AND_WEIGHTS/GSTENV2-S_weights')

In [ ]:
!mkdir /kaggle/working/results

## Plotting the graphs

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper right')
plt.savefig("/kaggle/working/results/loss.png")
plt.show()


In [ ]:

plt.plot(history.history['Caccuracy'])
plt.plot(history.history['val_Caccuracy'])
plt.title('model Caccuracy')
plt.ylabel('Caccuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper right')
plt.savefig("/kaggle/working/results/Caccuracy.png")
plt.show()


In [ ]:

plt.plot(history.history['Laccuracy'])
plt.plot(history.history['val_Laccuracy'])
plt.title('model Laccuracy')
plt.ylabel('Laccuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper right')
plt.savefig("/kaggle/working/results/Laccuracy.png")
plt.show()


In [ ]:

plt.plot(history.history['Daccuracy'])
plt.plot(history.history['val_Daccuracy'])
plt.title('model Daccuracy')
plt.ylabel('Daccuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper right')
plt.savefig("/kaggle/working/results/Daccuracy.png")
plt.show()

## Helper functions for making predictions

In [ ]:
cnt=0
def plotOBB(imgPath,all_labels):
    imgf=cv2.imread(imgPath,3)
    
    #Plotting all the labels
    imgOBB=[]
    for label in all_labels:
        angle=math.asin(label[-1])
        angle=math.degrees(angle)
        flg=1
        if angle<0:
            flg=-1
        angle=90*flg-angle
        
        box=[tuple([label[1]*IMAGE_SIZE[0],label[2]*IMAGE_SIZE[1]]),tuple([label[3]*IMAGE_SIZE[0],label[4]*IMAGE_SIZE[1]]),-angle]
        box=cv2.boxPoints(box)
        box=np.int0(box)
        imgOBB=cv2.drawContours(imgf,[box],-1,(0,0,255),2)
        font=cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(imgf,CLASSES[int(label[0])],(15,15),font,0.5,(0,0,255),2)
    
    plt.legend(CLASSES[int(all_labels[0][0])], loc='upper right')
    plt.savefig("/kaggle/working/results/"+str(cnt)+".jpg")
    plt.imshow(imgOBB)
    plt.show()

In [ ]:
def make_prediction(imgPath):
    NORMimg=np.float32(cv2.imread(imgPath))/255
    inpTensor=tf.convert_to_tensor([NORMimg])
    outputs=model.predict(inpTensor)
    predictedClasses=[]
    for i in range(0,outputs.shape[0]):
        index=tf.math.argmax(outputs[i][:totClasses])
        predictedClasses.append(index)
    index=[index.numpy()]
    #print("The class:",index) 
    label=[index[0]]+list(outputs[0][totClasses:])
    #print(label)
    plotOBB(imgPath,[label])

In [ ]:
'''
print("Predictions on training dataset")
path="/kaggle/working/Training"
files=os.listdir(path)
for file in files:
    s=file[-3:]
    if s=="txt":
        continue
    imgPath=os.path.join(path,file)
    make_prediction(imgPath)
'''

In [ ]:
'''
print("Predictions on validation dataset")
path="/kaggle/working/Validation"
files=os.listdir(path)
for file in files:
    s=file[-3:]
    if s=="txt":
        continue
    imgPath=os.path.join(path,file)
    make_prediction(imgPath)
'''